In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-05-04 14:06:38--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  86.6MB/s    in 0.3s    

2025-05-04 14:06:39 (86.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
# add your code here - consider creating a new cell for each section of code
# Count the number of ratings per user and per book
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Filter out users who have rated less than 200 books
filtered_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Filter out books that have fewer than 100 ratings
filtered_ratings = filtered_ratings[filtered_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

# Create a user-item matrix with users as rows and books (ISBNs) as columns
book_pivot = filtered_ratings.pivot_table(index='user', columns='isbn', values='rating', fill_value=0)

# Check the pivot table
book_pivot.head()


isbn,002542730X,0060008032,0060096195,006016848X,0060173289,0060175400,006019491X,0060199652,0060391626,0060392452,...,1558744630,1558745157,1559029838,1573225517,1573225789,1573227331,1573229326,1573229571,1592400876,1878424319
user,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
from sklearn.neighbors import NearestNeighbors

# Instantiate the KNN model with cosine distance metric
knn = NearestNeighbors(n_neighbors=6, metric='cosine')

# Fit the KNN model to the transposed pivot table (books are columns)
knn.fit(book_pivot.values.T)


NearestNeighbors(metric='cosine', n_neighbors=6)

In [8]:
def get_recommends(book_title):
    # Check if the book title exists in the dataframe
    book_isbn = df_books[df_books['title'] == book_title]['isbn'].values
    if len(book_isbn) == 0:
        return f"Book '{book_title}' not found in the dataset."

    book_isbn = book_isbn[0]

    # Get the index of the book in the pivot table
    book_idx = book_pivot.columns.get_loc(book_isbn)

    # Get the nearest neighbors for this book (excluding itself)
    distances, indices = knn.kneighbors(book_pivot.values.T[book_idx].reshape(1, -1))

    # List to hold the recommended books and their distances
    recommended_books = []

    for i in range(1, len(distances[0])):  # Start from 1 to avoid recommending the book itself
        # Get the ISBN of the recommended book
        recommended_isbn = book_pivot.columns[indices[0][i]]
        # Get the title of the recommended book
        recommended_title = df_books[df_books['isbn'] == recommended_isbn]['title'].values[0]
        recommended_books.append([recommended_title, distances[0][i]])

    return [book_title, recommended_books]

# Test the function
recommended_books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(recommended_books)


['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', np.float32(0.51784116)], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', np.float32(0.53763384)], ['Interview with the Vampire', np.float32(0.73450685)], ['The Witching Hour (Lives of the Mayfair Witches)', np.float32(0.74486566)], ['Catch 22', np.float32(0.7939835)]]]


In [9]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', np.float32(0.7234864)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Surgeon', np.float32(0.7699411)], ['The Weight of Water', np.float32(0.77085835)], ["I'll Be Seeing You", np.float32(0.8016211)]]]
You haven't passed yet. Keep trying!
